In [1]:
#Importing Packages I'll need
import pandas as pd
import numpy as np
import os
import scipy as sp
import re

# Initial Loading

In [ ]:
#Loading the main Opportunity Atlas file. This will only work if the file below is in the same folder as this notebook
OA = pd.read_csv("tract_outcomes_early.csv")
OA.head()

#### Loading the Business Master File Data.

In [3]:
#First, 1989
BMF_1989 = pd.read_csv("bmf.bmf89.csv")
BMF_1989.head()

/Users/katiawilliams/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (14,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,EIN,NAME,CITY,STATE,ZIP,SUBSECCD,ZIP5,MSA_NECH,PMSA,fipsold,...,Income,stateOld1,zip5Old1,cityOld1,NteeOldB,Ntee2007a,Confidence2007a,RandNum,nteeFinal,nteeFinal1
0,10024155,BANGOR BAND,BANGOR,ME,04401-5828,3.0,4401,733.0,NaN,NaN,...,0,ME,4401,BANGOR,Z99,A6C,A,0.36818,A6C,A
1,10024645,BANGOR SYMPHONY ORCHESTRA,BANGOR,ME,04401-1441,3.0,4401,733.0,NaN,NaN,...,0,ME,4401,BANGOR,Z99,A69,A,0.54812,A69,A
2,10078060,HENRIETTA D GOODALL HOSPITAL INC,SANFORD,ME,04073-2621,3.0,4073,NaN,NaN,NaN,...,0,ME,4073,SANFORD,Z99,E22,A,0.63606,E22,E
3,10130427,NORTHERN CUMBERLAND MEMORIAL HOSPITAL,BRIDGETON,ME,04009-0000,3.0,4009,6403.0,NaN,NaN,...,0,ME,4009,BRIDGETON,Z99,E22,A,0.53593,E22,E
4,10131950,OLD FOLKS HOME IN BATH,BATH,ME,04530-1697,3.0,4530,NaN,NaN,NaN,...,0,ME,4530,BATH,Z99,A80,C,0.77148,A80,A


In [4]:
#Next 2015. I've downloaded all the 2015 files and am combining them to make sure all nonprofits that began and ended
#in 2015 are included.

#Once you've run this cell, don't run it again

#This makes an empty table
BMF_2015 = pd.DataFrame([])

#This goes through the individual files and adds them to the table
for filename in os.listdir('2015_bmf'):
     BMF_2015 = BMF_2015.append(pd.read_csv('2015_bmf/'+filename, low_memory=False))

BMF_2015.head()

,ACCPER,ADDRESS,ASSETS,CITY,EIN,EPOST,FILER,FIPS,FISYR_IMAGE,FNDNCD,...,cFinSrc,cTaxPer,cTotRev,czFiler,nteeConf,nteeFinal,nteeFinal1,ntmaj10,ntmaj12,ntmaj5
0,6.0,1706 GREENLEAF DR,0.0,NORFOLK,3154.0,Z,N,51710.0,NaN,15.0,...,NaN,NaN,NaN,N,A,C36,C,EN,EN,OT
1,12.0,1050 THORNDIKE ST,0.0,PALMER,19818.0,Z,N,25013.0,NaN,10.0,...,NaN,NaN,NaN,N,A,X21,X,RE,RE,OT
2,12.0,523 E BROADWAY,0.0,SOUTH BOSTON,29215.0,Z,N,25025.0,NaN,10.0,...,NaN,NaN,NaN,N,C,X20,X,RE,RE,OT
3,12.0,PO BOX 92,0.0,HOSFORD,260049.0,Z,N,12077.0,NaN,10.0,...,NaN,NaN,NaN,N,A,X21,X,RE,RE,OT
4,12.0,PO BOX 296,0.0,LABELLE,490336.0,Z,N,12051.0,NaN,10.0,...,NaN,NaN,NaN,N,A,X21,X,RE,RE,OT


In [5]:
#Since the same org will appear several times, this will get rid of duplicates
#BMF_2015.drop_duplicates(inplace = True)

BMF_2015_unique = BMF_2015.groupby("EIN").first()
BMF_2015_unique.head()

,ACCPER,ADDRESS,ASSETS,CITY,EPOST,FILER,FIPS,FISYR_IMAGE,FNDNCD,FRCD,...,cFinSrc,cTaxPer,cTotRev,czFiler,nteeConf,nteeFinal,nteeFinal1,ntmaj10,ntmaj12,ntmaj5
EIN,,,,,,,,,,,,,,,,,,,,,
3154.0,6.0,1706 GREENLEAF DR,0.0,NORFOLK,Z,N,51710.0,NaN,15.0,2.0,...,NaN,NaN,NaN,N,A,C36,C,EN,EN,OT
19818.0,12.0,1050 THORNDIKE ST,0.0,PALMER,Z,N,25013.0,NaN,10.0,6.0,...,NaN,NaN,NaN,N,A,X21,X,RE,RE,OT
29215.0,12.0,523 E BROADWAY,0.0,SOUTH BOSTON,Z,N,25025.0,NaN,10.0,6.0,...,NaN,NaN,NaN,N,C,X20,X,RE,RE,OT
65837.0,9.0,4655 STATE ROUTE 60,0.0,WAKEMAN,Z,N,39077.0,NaN,0.0,1.0,...,NaN,NaN,NaN,N,A,K28,K,HU,HU,HU
260049.0,12.0,PO BOX 92,0.0,HOSFORD,Z,N,12077.0,NaN,10.0,6.0,...,NaN,NaN,NaN,N,A,X21,X,RE,RE,OT


#### The Business Master File is more granular than Opportunity Atlas, so we have to match BMF to the minimum shared granularity.

I originally wanted to do census tract, since this is the actual minimum shared granularity. However, since the 1989 BMF data does not include addresses, we cannot match the areas to census tracts easily. The next best thing is to work on the city level, and instead match opportutnity atlas data to cities.

In [6]:
city_tract = pd.io.stata.read_stata('city_name_to_tract.dta')
city_tract.head()

,state,county,tract,city,share_tract_in_city,maxval,max,state_abbr
0,1,1,20100,Prattville,1.0,1.0,1.0,AL
1,1,1,20200,Prattville,1.0,1.0,1.0,AL
2,1,1,20300,Prattville,1.0,1.0,1.0,AL
3,1,1,20400,Prattville,1.0,1.0,1.0,AL
4,1,1,20500,Prattville,1.0,1.0,1.0,AL


In [54]:
OA_city = OA.merge(city_tract, "left")

From here, I'm going to re-aggregate all the means according to city instead of census tract. I'll have to:
* Take each county mean value
* Multiply it by that mean's count or n (which gives the sum)
* Sum the expanded mean (now a sum) and the count by city
* Divide "summed sums" by "summed counts"

In [59]:
len(OA_city[pd.Series([pd.isnull(x) for x in OA_city['city']])].iloc[:,0])

3689

~3,700 tracts aren't matched to cities :( sad

In [60]:
len(OA_city.iloc[:,0])

73278

But there are 73,000 entires, so maybe 69,000 left after those without cities are taken out.

Meh...I'll just remove them for now.

In [66]:
OA_city_noNA = OA_city[pd.Series([pd.notnull(x) for x in OA_city['city']])]

In [68]:
OA_city_noNA.columns

Index(['state', 'county', 'tract', 'kir_natam_female_p1',
       'kir_natam_female_p25', 'kir_natam_female_p50', 'kir_natam_female_p75',
       'kir_natam_female_p100', 'kir_natam_female_n', 'kir_natam_female_mean',
       ...
       'working_white_pooled_mean_se', 'kir_stycz_white_pooled_mean_se',
       'kfr_stycz_white_pooled_mean_se', 'spouse_rk_white_pooled_mean_se',
       'par_rank_white_pooled_mean_se', 'city', 'share_tract_in_city',
       'maxval', 'max', 'state_abbr'],
      dtype='object', length=7830)

Possibly there's a way to pool median/ percentile values but I'm not sure of it and we have plenty of averages so I'll use those.

There's also a way to pool SEs, but we'd be assuming that each tract's mean values are independent from each other. 

First I'll just pool the means :)

In [84]:
[x for x in OA_city_noNA.columns if "kid" in x]

['kid_hisp_male_n',
 'kid_hisp_female_n',
 'kid_white_male_n',
 'kid_white_female_n',
 'kid_black_male_n',
 'kid_black_female_n',
 'kid_natam_male_n',
 'kid_natam_female_n',
 'kid_asian_male_n',
 'kid_asian_female_n',
 'kid_pooled_pooled_n',
 'kid_pooled_male_n',
 'kid_pooled_female_n',
 'kid_white_pooled_n',
 'kid_black_pooled_n',
 'kid_asian_pooled_n',
 'kid_hisp_pooled_n',
 'kid_natam_pooled_n',
 'kid_pooled_pooled_blw_p50_n',
 'kid_white_pooled_blw_p50_n',
 'kid_black_pooled_blw_p50_n',
 'kid_hisp_pooled_blw_p50_n',
 'kid_asian_pooled_blw_p50_n',
 'kid_natam_pooled_blw_p50_n',
 'kid_pooled_male_blw_p50_n',
 'kid_white_male_blw_p50_n',
 'kid_black_male_blw_p50_n',
 'kid_hisp_male_blw_p50_n',
 'kid_asian_male_blw_p50_n',
 'kid_natam_male_blw_p50_n',
 'kid_pooled_female_blw_p50_n',
 'kid_white_female_blw_p50_n',
 'kid_black_female_blw_p50_n',
 'kid_hisp_female_blw_p50_n',
 'kid_asian_female_blw_p50_n',
 'kid_natam_female_blw_p50_n']

In [94]:
#All the columns that contain mean values
mean_col_names = [x for x in OA_city_noNA.columns if ("se" not in x) & ("_mean" in x) & ("par_rank_other" not in x)]
mean_col_names[0:5]

OA_tract_city = OA_city_noNA[['city', 'tract','state_abbr']]

#Sanity check that each corresponds to a column with a count in it
prefixes = np.array([])
for col in mean_col_names:
    pre = re.search(r"(.*)_mean$", col).group(1)
    prefixes = np.append(prefixes, pre)
    
    # The "par_rank" columns correspond to "kid" counts so some more maneuvering is needed
    if ("par_rank" in pre) & ("other" not in pre):
        suf = re.search(r"par_rank_(.*)", pre).group(1)
        OA_tract_city[pre + "_n"] = OA_city_noNA["kid_" + suf + "_n"]
        OA_tract_city[pre + "_sum"] = OA_city_noNA[col]*OA_city_noNA["kid_" + suf + "_n"]
    elif (pre + "_n") not in OA_city_noNA.columns:
        print(pre)
    else:
        OA_tract_city[pre + "_n"] = OA_city_noNA[pre + "_n"]
        OA_tract_city[pre + "_sum"] = OA_city_noNA[pre + "_n"]* OA_city_noNA[col]

/Users/katiawilliams/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/katiawilliams/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/katiawilliams/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

In [96]:
OA_grouped = OA_tract_city.groupby(['city', 'state_abbr']).sum()

#At this point the tract value doesn't make sense, so we'll remove it
OA_grouped.drop("tract", inplace = True, axis=1)
OA_grouped.head()

kir_natam_female_n  kir_natam_female_sum  \
city      state_abbr                                             
Abbeville AL                         NaN                   NaN   
          GA                         NaN                   NaN   
          LA                         NaN                   NaN   
          MS                         NaN                   NaN   
          SC                         NaN                   NaN   

                      jail_natam_female_n  jail_natam_female_sum  \
city      state_abbr                                               
Abbeville AL                          NaN                    NaN   
          GA                          NaN                    NaN   
          LA                          NaN                    NaN   
          MS                          NaN                    NaN   
          SC                          NaN                    NaN   

                      kfr_natam_female_n  kfr_natam_female_sum  \
city      state_abbr                                             
Abbeville AL                         NaN                   NaN   
          GA                         NaN                   NaN   
          LA                         NaN                   NaN   
          MS                         NaN                   NaN   
          SC                         NaN                   NaN   

                      kfr_top20_natam_female_n  kfr_top20_natam_female_sum  \
city      state_abbr                                                         
Abbeville AL                               NaN                         NaN   
          GA                               NaN                         NaN   
          LA                               NaN                         NaN   
          MS                               NaN                         NaN   
          SC                               NaN                         NaN   

                      kir_top20_natam_female_n  kir_top20_natam_female_sum  \
city      state_abbr                                                         
Abbeville AL                               NaN                         NaN   
          GA                               NaN                         NaN   
          LA                               NaN                         NaN   
          MS                               NaN                         NaN   
          SC                               NaN                         NaN   

                                ...             marr_32_white_pooled_n  \
city      state_abbr            ...                                      
Abbeville AL                    ...                              511.0   
          GA                    ...                              148.0   
          LA                    ...                             2051.0   
          MS                    ...                              177.0   
          SC                    ...                              653.0   

                      marr_32_white_pooled_sum  work_32_white_pooled_n  \
city      state_abbr                                                     
Abbeville AL                        285.492250                   511.0   
          GA                         82.832470                   148.0   
          LA                       1071.257894                  2051.0   
          MS                         85.556465                   177.0   
          SC                        385.433653                   653.0   

                      work_32_white_pooled_sum  work_26_white_pooled_n  \
city      state_abbr                                                     
Abbeville AL                        392.940200                   449.0   
          GA                        101.214614                   129.0   
          LA                       1572.804732                  1759.0   
          MS                        127.911834                   149.0   
          SC                        515.532

In [99]:
for p in prefixes:
    n = OA_grouped[p + "_n"]
    s = OA_grouped[p + "_sum"]
    OA_grouped[p + "_mean"] = s/n
OA_grouped.head()

kir_natam_female_n  kir_natam_female_sum  \
city      state_abbr                                             
Abbeville AL                         NaN                   NaN   
          GA                         NaN                   NaN   
          LA                         NaN                   NaN   
          MS                         NaN                   NaN   
          SC                         NaN                   NaN   

                      jail_natam_female_n  jail_natam_female_sum  \
city      state_abbr                                               
Abbeville AL                          NaN                    NaN   
          GA                          NaN                    NaN   
          LA                          NaN                    NaN   
          MS                          NaN                    NaN   
          SC                          NaN                    NaN   

                      kfr_natam_female_n  kfr_natam_female_sum  \
city      state_abbr                                             
Abbeville AL                         NaN                   NaN   
          GA                         NaN                   NaN   
          LA                         NaN                   NaN   
          MS                         NaN                   NaN   
          SC                         NaN                   NaN   

                      kfr_top20_natam_female_n  kfr_top20_natam_female_sum  \
city      state_abbr                                                         
Abbeville AL                               NaN                         NaN   
          GA                               NaN                         NaN   
          LA                               NaN                         NaN   
          MS                               NaN                         NaN   
          SC                               NaN                         NaN   

                      kir_top20_natam_female_n  kir_top20_natam_female_sum  \
city      state_abbr                                                         
Abbeville AL                               NaN                         NaN   
          GA                               NaN                         NaN   
          LA                               NaN                         NaN   
          MS                               NaN                         NaN   
          SC                               NaN                         NaN   

                                ...              kir_24_white_pooled_mean  \
city      state_abbr            ...                                         
Abbeville AL                    ...                              0.482208   
          GA                    ...                              0.479642   
          LA                    ...                              0.476599   
          MS                    ...                              0.414211   
          SC                    ...                              0.544885   

                      marr_26_white_pooled_mean  marr_29_white_pooled_mean  \
city      state_abbr                                                         
Abbeville AL                           0.445968                   0.526599   
          GA                           0.340178                   0.469560   
          LA                           0.410694                   0.501098   
          MS                           0.431940                   0.534298   
          SC                           0.443972                   0.577350   

                      kfr_29_white_pooled_mean  kir_29_white_pooled_mean  \
city      state_abbr                                                       
Abbeville AL                          0.527224                  0.494739   
          GA                          0.466506                  0.451268   
          LA                          0.577534                  0.535558   
          MS                          0.477329            

In [100]:
BMF_2015.columns

Index(['ACCPER', 'ADDRESS', 'ASSETS', 'CITY', 'EIN', 'EPOST', 'FILER', 'FIPS',
       'FISYR_IMAGE', 'FNDNCD', 'FRCD', 'GEN', 'INCOME', 'IRS990n', 'LEVEL1',
       'LEVEL2', 'LEVEL3', 'LEVEL4', 'MAJGRPB', 'MSA_NECH', 'NAICS', 'NAME',
       'NTEE1', 'NTEECC', 'NTEESRC', 'OUTNCCS', 'OUTREAS', 'PFFRCD', 'PMSA',
       'RULEDATE', 'RandNum', 'SEC_NAME', 'STATE', 'SUBSECCD', 'TAXPER',
       'Unnamed: 0', 'ZFILER', 'ZIP5', 'cAssets', 'cFiler', 'cFinSrc',
       'cTaxPer', 'cTotRev', 'czFiler', 'nteeConf', 'nteeFinal', 'nteeFinal1',
       'ntmaj10', 'ntmaj12', 'ntmaj5'],
      dtype='object')

In [101]:
BMF_1989.columns

Index(['EIN', 'NAME', 'CITY', 'STATE', 'ZIP', 'SUBSECCD', 'ZIP5', 'MSA_NECH',
       'PMSA', 'fipsold', 'FIPS', 'FNDNCD', 'ORGCD', 'ORGTYPCD', 'v_errorCd',
       'FILER', 'NTEECC', 'NteeCC4d', 'NTEEconf', 'NTEE1', 'LEVEL1', 'LEVEL2',
       'LEVEL3', 'LEVEL4', 'MAJGRPB', 'OUTNCCS', 'OUTREAS', 'NTEESRC',
       'ntmaj10', 'ntmaj12', 'ntmaj5', 'Assets', 'Income', 'stateOld1',
       'zip5Old1', 'cityOld1', 'NteeOldB', 'Ntee2007a', 'Confidence2007a',
       'RandNum', 'nteeFinal', 'nteeFinal1'],
      dtype='object')

In [102]:
overlap = [x for x in BMF_2015.columns if x in BMF_1989.columns]
overlap

['CITY',
 'EIN',
 'FILER',
 'FIPS',
 'FNDNCD',
 'LEVEL1',
 'LEVEL2',
 'LEVEL3',
 'LEVEL4',
 'MAJGRPB',
 'MSA_NECH',
 'NAME',
 'NTEE1',
 'NTEECC',
 'NTEESRC',
 'OUTNCCS',
 'OUTREAS',
 'PMSA',
 'RandNum',
 'STATE',
 'SUBSECCD',
 'ZIP5',
 'nteeFinal',
 'nteeFinal1',
 'ntmaj10',
 'ntmaj12',
 'ntmaj5']

In [103]:
BMF_2015_clean = BMF_2015[overlap]
BMF_1989_clean = BMF_1989[overlap]

## Write to computer

Lastly, we write the new files to the folder so we have cleaned versions to work on later

In [104]:
#OA_grouped.to_csv("Opp_At_cleaned.csv")
#BMF_1989_clean.to_csv("BMF_1989_cleaned.csv")
#BMF_2015_clean.to_csv("BMF_2015_cleaned.csv")